In [1]:
import pandas as pd

C:\Users\psapkota\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
df1 = pd.read_csv("listings.csv")

In [16]:
df2 = df1[df1["review_scores_location"] > 4.9]
df = df2[df2["review_scores_rating"] < 4.0]
print(df[["id","review_scores_location","review_scores_rating"]])

                 id  review_scores_location  review_scores_rating
119    3.567190e+05                     5.0                  0.00
269    6.028770e+05                     5.0                  3.75
1763   1.137731e+07                     5.0                  3.00
1878   1.067162e+07                     5.0                  1.00
3306   2.129219e+07                     5.0                  3.00
...             ...                     ...                   ...
17342  5.408035e+07                     5.0                  3.50
17691  5.819680e+17                     5.0                  3.00
17710  5.227936e+07                     5.0                  3.00
18061  1.436002e+07                     5.0                  3.67
18273  4.796882e+07                     5.0                  3.00

[65 rows x 3 columns]


In [17]:
df["hex_id"] = df.apply(
    func=lambda row: h3.geo_to_h3(lat=row["latitude"], lng=row["longitude"], resolution=7), axis=1)

<ipython-input-17-3f61fce7f3ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hex_id"] = df.apply(


In [18]:
df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,hex_id
119,3.567190e+05,https://www.airbnb.com/rooms/356719,2.022090e+13,9/13/2022,previous scrape,Cozy and Modern Home,"Hello and thank you for checking out our cozy,...",We love that we are so close to all Austin att...,https://a0.muscache.com/pictures/3975228/46e90...,1805524,...,5.0,5.0,NaN,f,1,1,0,0,0.01,87489e353ffffff
269,6.028770e+05,https://www.airbnb.com/rooms/602877,2.022090e+13,9/13/2022,city scrape,Stylish Furnished Condo with Pool,"Looking for a long-term renter, ideally Jan. 1...","Central Austin has great restaurants, and ente...",https://a0.muscache.com/pictures/8471892/4eea1...,1280071,...,5.0,4.0,NaN,f,1,1,0,0,0.04,87489e35bffffff
1763,1.137731e+07,https://www.airbnb.com/rooms/11377313,2.022090e+13,9/13/2022,previous scrape,Private room in cozy duplex,"SXSW availability:<br />Our home is warm, clea...",NaN,https://a0.muscache.com/pictures/df4dbef1-55f1...,59646051,...,5.0,3.0,NaN,f,1,0,1,0,0.01,87489e245ffffff
1878,1.067162e+07,https://www.airbnb.com/rooms/10671623,2.022090e+13,9/13/2022,previous scrape,"Farmhouse In The City, Your SXSW Paradise","This quaint home was built in the 1800's, and ...",This is the greatest neighborhood I have ever ...,https://a0.muscache.com/pictures/0129d898-09fc...,53170299,...,5.0,1.0,NaN,f,1,0,1,0,0.01,87489e370ffffff
3306,2.129219e+07,https://www.airbnb.com/rooms/21292194,2.022090e+13,9/13/2022,previous scrape,Modern South Austin Studio,"Chic and modern little concrete bungalow, stud...",NaN,https://a0.muscache.com/pictures/9d4766e6-1ded...,46983898,...,5.0,4.0,NaN,f,1,1,0,0,0.02,87489e362ffffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17342,5.408035e+07,https://www.airbnb.com/rooms/54080351,2.022090e+13,9/13/2022,city scrape,VILLA 2101 - Island Resort - Panoramic Lakevie...,** NEW LISTING ** Beautifully remodeled lakefr...,The Island on Lake Travis,https://a0.muscache.com/pictures/prohost-api/H...,32760529,...,5.0,4.5,NaN,t,26,26,0,0,0.44,874898cf3ffffff
17691,5.819680e+17,https://www.airbnb.com/rooms/581968387262052587,2.022090e+13,9/13/2022,city scrape,Master bedroom,Take a break and unwind at this peaceful oasis.,NaN,https://a0.muscache.com/pictures/c58e42b5-0254...,448763312,...,5.0,1.0,NaN,f,4,0,4,0,0.17,87489e21effffff
17710,5.227936e+07,https://www.airbnb.com/rooms/52279358,2.022090e+13,9/13/2022,city scrape,NEW! Bright Lago Vista Home - Walk to Lake Tra...,Escape from the city and enjoy a breath of fre...,"OUTDOOR REC: Lake Travis (0.4 miles), Emerald ...",https://a0.muscache.com/pictures/prohost-api/H...,124060715,...,5.0,3.0,NaN,t,147,147,0,0,0.11,874898cf0ffffff
18061,1.436002e+07,https://www.airbnb.com/rooms/14360020,2.022090e+13,9/13/2022,city scrape,"Marble Falls, TX, 2-Bedroom Presidential #1",I can accept a 1-night stay for a weekend nigh...,Break away from everyday life and retreat to c...,https://a0.muscache.com/pictures/576d80c5-5b12...,10981379,...,5.0,4.0,NaN,f,57,57,0,0,0.05,8748981acffffff


In [19]:
grouped_q5 = df.groupby(by="hex_id")['id'].count()
grouped_q5

hex_id
874898005ffffff    1
87489800affffff    1
8748980c4ffffff    1
8748981acffffff    2
874898c99ffffff    1
874898cacffffff    1
874898cf0ffffff    1
874898cf3ffffff    1
874898d52ffffff    1
87489a96dffffff    1
87489e21effffff    1
87489e220ffffff    1
87489e22bffffff    1
87489e245ffffff    2
87489e246ffffff    1
87489e25effffff    1
87489e262ffffff    1
87489e265ffffff    1
87489e271ffffff    4
87489e2a6ffffff    1
87489e2c4ffffff    1
87489e340ffffff    3
87489e341ffffff    2
87489e342ffffff    3
87489e344ffffff    2
87489e345ffffff    1
87489e346ffffff    9
87489e353ffffff    1
87489e355ffffff    1
87489e358ffffff    2
87489e35affffff    1
87489e35bffffff    1
87489e35cffffff    1
87489e362ffffff    3
87489e370ffffff    3
87489e371ffffff    2
87489ea0dffffff    1
87489ead9ffffff    2
87489eae4ffffff    1
Name: id, dtype: int64

In [20]:
grouped_q5 = grouped_q5.reset_index()
grouped_q5
# grouped_q5.sample(n=5, random_state=42)

,hex_id,id
0,874898005ffffff,1
1,87489800affffff,1
2,8748980c4ffffff,1
3,8748981acffffff,2
4,874898c99ffffff,1
5,874898cacffffff,1
6,874898cf0ffffff,1
7,874898cf3ffffff,1
8,874898d52ffffff,1
9,87489a96dffffff,1


In [21]:
map = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()],
                 zoom_start=12,  # 8.5 after a bit of experimentation, we thought this presents the map best
                 tiles="cartodbpositron")
map

In [22]:
# !pip install geojson
import json
import geojson


def to_geojson(row):
    """Transform hex_id into a geojson object."""
    geometry = {
        "type": "Polygon",
        "coordinates": [h3.h3_to_geo_boundary(h=row["hex_id"], geo_json=True)]
    }
    return geojson.Feature(id=row["hex_id"], geometry=geometry, properties={"id": row["id"]})


geojsons = grouped_q5.apply(func=to_geojson, axis=1).values.tolist()
geojson_str: str = json.dumps(geojson.FeatureCollection(geojsons))

In [23]:
import matplotlib.cm

# instantiate a colormap object for better visualisation
colormap = matplotlib.cm.get_cmap(name="plasma")
max_id = grouped_q5["id"].max()
min_id = grouped_q5["id"].min()

In [24]:
_ = folium.GeoJson(data=geojson_str, style_function=lambda f: {
    "fillColor": matplotlib.colors.to_hex(
        colormap((f["properties"]["id"] - min_id) / (max_id - min_id))),
    "color": "black",
    "weight": 0,
    "fillOpacity": 0.3
}).add_to(map)
map